# Teacher tool

### Imports

In [17]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 2.9 MB/s eta 0:00:00


In [18]:
!mkdir reports

In [2]:
import json
import os
import gradio as gr
import pandas as pd
import subprocess
import re
import sys

## Load and Save Dataset Functions


# Adjust the paths in the following cell

In [3]:
csv_file = "/content/drive/MyDrive/IILAP CODE/data_annotation_new_2.csv"
credibility_file = "/content/drive/MyDrive/IILAP CODE/source_credibility_2 (1).json"
dataset = []
source_credibility = {}

In [4]:
def load_dataset():
    global dataset
    if os.path.isfile(csv_file):
        try:
            df = pd.read_csv(csv_file, encoding="utf-8")
        except UnicodeDecodeError:
            try:
                df = pd.read_csv(csv_file, encoding="utf-8-sig")
            except UnicodeDecodeError:
                df = pd.read_csv(csv_file, encoding="ISO-8859-1")
        dataset[:] = df.to_dict(orient="records")
    else:
        dataset = []

def save_dataset():
    df = pd.DataFrame(dataset)
    df.to_csv(csv_file, index=False, quoting=1)

## Source Functions

In [5]:
def load_credibility():
    global source_credibility
    if os.path.isfile(credibility_file):
        with open(credibility_file, "r", encoding="utf-8") as f:
            source_credibility = json.load(f)
    else:
        source_credibility = {}

In [6]:
def save_credibility():
    with open(credibility_file, "w", encoding="utf-8") as f:
        json.dump(source_credibility, f, indent=4, ensure_ascii=False)

In [7]:
def get_credibility_table():
    df = pd.DataFrame(list(source_credibility.items()), columns=["Source", "Credibility"])
    return df


In [8]:
def add_or_update_source(name, level):
    if level not in ["Low", "Medium", "High"]:
        return "\u26a0\ufe0f Credibility must be 'Low', 'Medium', or 'High'", get_credibility_table()
    source_credibility[name.strip()] = level
    save_credibility()
    return f"✅ Source '{name}' added/updated.", get_credibility_table()

In [9]:
def delete_source(name):
    name = name.strip()
    if name in source_credibility:
        del source_credibility[name]
        save_credibility()
        return f"🗑️ Source '{name}' deleted.", get_credibility_table()
    return "\u26a0\ufe0f Source not found.", get_credibility_table()

## Data functions

In [10]:
def save_simple_annotations(q, a, true_parts, true_sources, true_names, false_parts, false_sources, false_names, issues):
    true_parts_list = [p.strip() for p in true_parts.strip().splitlines() if p.strip()]
    true_sources_list = [s.strip() for s in true_sources.strip().splitlines() if s.strip()]
    true_names_list = [n.strip() for n in true_names.strip().splitlines() if n.strip()]
    false_parts_list = [p.strip() for p in false_parts.strip().splitlines() if p.strip()]
    false_sources_list = [s.strip() for s in false_sources.strip().splitlines() if s.strip()]
    false_names_list = [n.strip() for n in false_names.strip().splitlines() if n.strip()]

    entry = {
        "question": q.strip(),
        "answer": a.strip(),
        "confirmed_parts": json.dumps(true_parts_list, ensure_ascii=False),
        "confirmed_sources": json.dumps(true_sources_list, ensure_ascii=False),
        "false_claim_parts": json.dumps(false_parts_list, ensure_ascii=False),
        "false_claim_sources": json.dumps(false_sources_list, ensure_ascii=False),
        "confirmed_sources_names": json.dumps(true_names_list, ensure_ascii=False),
        "fake_sources_names": json.dumps(false_names_list, ensure_ascii=False),
        "annotation_issues": json.dumps(issues, ensure_ascii=False),
    }

    dataset.append(entry)
    save_dataset()
    df = pd.DataFrame(dataset)
    return "✅ Entry saved!", df.to_csv(index=False, quoting=1), df


In [11]:
def delete_row(index):
    index = int(index)
    if 0 <= index < len(dataset):
        del dataset[index]
        save_dataset()
        df = pd.DataFrame(dataset).reset_index().rename(columns={"index": "Row Index"})
        return f"🗑️ Deleted row {index + 1}", df.to_csv(index=False, quoting=1), df
    else:
        df = pd.DataFrame(dataset).reset_index().rename(columns={"index": "Row Index"})
        return "\u26a0\ufe0f Invalid index", df.to_csv(index=False, quoting=1), df


## Launching the student interface

# Adjust the path in the following cell




In [12]:
def launch_student_interface():
    proc = subprocess.Popen([sys.executable, "-u", "/content/drive/MyDrive/IILAP CODE/student_interface.py"], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, bufsize=1)
    public_url = None
    while True:
        line = proc.stdout.readline()
        if not line:
            break
        decoded_line = line.decode("utf-8").strip()
        match = re.search(r'(https?://\S*gradio\.live\S*)', decoded_line)
        if match:
            public_url = match.group(1)
            break
    if public_url:
        return f'''
            <div>
                <a href="{public_url}" target="_blank" style="font-size:16px; font-weight:bold; color:#007bff;">Click to continue</a>
                <br><br>
                <input type="text" value="{public_url}" style="width:100%; font-size:14px; padding:8px;" readonly onclick="this.select(); document.execCommand('copy');" />
                <small style="color:gray;">(Click the link above or copy this URL)</small>
            </div>
        '''
    else:
        return "Failed to capture student interface URL."


## Evaluations

# Adjust the path

In [13]:

def run_evaluation(c1, c2, c3, c4, c5,c6):
    coefs = [c1, c2, c3, c4, c5,c6]

    # Check if all inputs are floats and >= 0
    try:
        coefs = [float(c) for c in coefs]
    except Exception:
        return "⚠️ Please enter valid decimal numbers for all coefficients.", ""

    if any(c < 0 for c in coefs):
        return "⚠️ Coefficients must be non-negative.", ""

    total = sum(coefs)
    if abs(total - 100.0) > 1e-6:
        return f"⚠️ The total of coefficients must be exactly 100%. Currently: {total}%.", ""

    eval_df = pd.DataFrame({
        "coef1": [coefs[0]],
        "coef2": [coefs[1]],
        "coef3": [coefs[2]],
        "coef4": [coefs[3]],
        "coef5": [coefs[4]],
        "coef6": [coefs[5]],
    })
    eval_csv_path = "/content/drive/MyDrive/IILAP CODE/evaluation_input.csv"
    eval_df.to_csv(eval_csv_path, index=False)

    try:
        proc = subprocess.run([sys.executable, "/content/drive/MyDrive/IILAP CODE/evaluation (1).py"], capture_output=True, text=True, timeout=30)
        output = proc.stdout
        err = proc.stderr
        if err:
            output += "\n[Error output]\n" + err
    except Exception as e:
        return f"⚠️ Failed to run evaluation.py: {e}", ""

    return "✅ Evaluation completed.", output



## Interface functions

In [14]:
load_dataset()
load_credibility()

In [15]:
with gr.Blocks(theme=gr.themes.Soft(), css="textarea { font-size: 15px !important }") as demo:
    gr.Markdown("""
<h1 style="text-align: center;">📚 Simulated Chat Bot Teacher Tool</h1>
<p style="text-align: center; font-size: 16px;">
Manage chatbot data, verify annotations, assess source credibility, deploy a student-facing interface, and run evaluations—all in one place.
</p>
<hr>
""")

    with gr.Row():
        show_add = gr.Button("✍️ Add New Annotation")
        show_data = gr.Button("📋 Existing Data")
        show_cred = gr.Button("📊 Source Credibility")
        show_student = gr.Button("🚀 Launch Student Interface")
        show_eval = gr.Button("🧮 Evaluation")  # New button


    with gr.Column(visible=True) as add_block:
        with gr.Group():
            with gr.Row():
                question = gr.Textbox(label="🧠 Question", lines=2)
                answer = gr.Textbox(label="📄 Full Answer", lines=4)

            with gr.Row():
                with gr.Column():
                    gr.Markdown("### ✅  Sentence(s) to be highlighted green")
                    true_parts = gr.Textbox(label="✔️ Sentences", lines=2)
                    true_sources = gr.Textbox(label="🔗 Source URLs", lines=2)
                    true_names = gr.Textbox(label="📚 Source Names", lines=2)
                with gr.Column():
                    gr.Markdown("### ❌ Sentence(s) to be highlighted red ")
                    false_parts = gr.Textbox(label="❌  Sentences", lines=2)
                    false_sources = gr.Textbox(label="🔗  Source URLs", lines=2)
                    false_names = gr.Textbox(label="📕  Source Names", lines=2)



            issue_options = [
                "Text is not highlighted", "True data labeled as true", "False data labeled as false",
                "Source link missmatch", "Good source linking", "False data appears in unhighlighted text",
                "Source appears but is not highlighted in text", "True data labeled as false", "False data labeled as true", "Wrong source credibility score","Correct source credibility score"
            ]
            issues = gr.CheckboxGroup(label="⚠\ufe0f Row categories", choices=issue_options)

            save_btn = gr.Button("📏 Save Row")
            status = gr.Textbox(label="📝 Status", interactive=False)

    with gr.Column(visible=False) as data_block:
        csv_output = gr.Textbox(label="📄 CSV Preview", lines=10, interactive=False, visible=False)
        with gr.Row():
            delete_index = gr.Number(label="🗑️ Row Index to Delete", value=0, precision=0)
            delete_btn = gr.Button("🗑️ Delete Row")
        table = gr.Dataframe(label="🗾 Current Dataset", interactive=False, wrap=True)

    with gr.Column(visible=False) as student_block:
        launch_btn = gr.Button("Launch Student Interface")
        student_output = gr.HTML()
    with gr.Column(visible=False) as eval_block:
        gr.Markdown("### Adjust Score Coefficients (%of total score)")
        with gr.Row():
            coef1 = gr.Number(label="Number of clicks 1+ ", value=20, precision=2)
            coef2 = gr.Number(label="Total number of questions 8+ ", value=20, precision=2)
            coef3 = gr.Number(label="More than 10 s spent on source ", value=20, precision=2)
            coef4 = gr.Number(label="More than 20s between questions ", value=20, precision=2)
            coef5 = gr.Number(label="Quiz score", value=10, precision=2)
            coef6 = gr.Number(label="Correctly identified errors ", value=10, precision=2)
        run_eval_btn = gr.Button("▶️ Run Evaluation")
        eval_status = gr.Textbox(label="Evaluation Status", interactive=False)
        eval_output = gr.Textbox(label="Evaluation Output", lines=3, interactive=False)

    with gr.Column(visible=False) as credibility_block:
        gr.Markdown(
    "### 🔍 Manage Source Credibility\n\n"
    "Preview all sources from the dataset. Add, update, or remove sources and their credibility levels here. "
    "(All sources not in this list will be labeled as having unknown credibility.)"
)
        with gr.Row():
            source_name = gr.Textbox(label="🔠 Source Name")
            credibility_level = gr.Dropdown(["Low", "Medium", "High"], label="🎯 Credibility Level")

        with gr.Row():
            add_update_btn = gr.Button("➕ Add / Update")
            delete_source_btn = gr.Button("❌ Delete")

        cred_status = gr.Textbox(label="📝 Status", interactive=False)
        credibility_table = gr.Dataframe(label="📋 Credibility List", interactive=False, wrap=True)

    def toggle_section(name):
        return (
        gr.update(visible=(name == "add")),
        gr.update(visible=(name == "data")),
        gr.update(visible=(name == "cred")),
        gr.update(visible=(name == "student")),
        gr.update(visible=(name == "eval")),
    )





    show_add.click(fn=lambda: toggle_section("add"), inputs=[], outputs=[add_block, data_block, credibility_block, student_block, eval_block])
    show_data.click(fn=lambda: toggle_section("data"), inputs=[], outputs=[add_block, data_block, credibility_block, student_block, eval_block])
    show_cred.click(fn=lambda: toggle_section("cred"), inputs=[], outputs=[add_block, data_block, credibility_block, student_block, eval_block])
    show_student.click(fn=lambda: toggle_section("student"), inputs=[], outputs=[add_block, data_block, credibility_block, student_block, eval_block])
    show_eval.click(fn=lambda: toggle_section("eval"), inputs=[], outputs=[add_block, data_block, credibility_block, student_block, eval_block])

    save_btn.click(save_simple_annotations,
        inputs=[question, answer, true_parts, true_sources, true_names, false_parts, false_sources, false_names, issues],
        outputs=[status, csv_output, table])

    delete_btn.click(delete_row, inputs=[delete_index], outputs=[status, csv_output, table])
    launch_btn.click(fn=launch_student_interface, inputs=[], outputs=[student_output])

    add_update_btn.click(fn=add_or_update_source,
                         inputs=[source_name, credibility_level],
                         outputs=[cred_status, credibility_table])

    delete_source_btn.click(fn=delete_source,
                            inputs=[source_name],
                            outputs=[cred_status, credibility_table])
    run_eval_btn.click(run_evaluation,inputs=[coef1, coef2, coef3, coef4, coef5,coef6],outputs=[eval_status, eval_output])


    demo.load(    lambda: (        "",         pd.DataFrame(dataset).reset_index().to_csv(index=False, quoting=1),
        pd.DataFrame(dataset).reset_index().rename(columns={"index": "Row Index"}),         get_credibility_table()    ),    inputs=[],
    outputs=[status, csv_output, table, credibility_table]        )

if __name__ == "__main__":
    demo.launch(server_name="0.0.0.0", server_port=7892)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://db1152de6632cbb0f3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
